In [2]:
import threading
import time
import math
import os
import cv2
import mediapipe as mp
import pyautogui
import numpy as np
from collections import deque
import tkinter as tk
from PIL import Image, ImageTk
import queue
import ctypes

# ===================== EMBEDDED PRO MouseModeEngine =====================
# Optimized Mouse Mode engine (embedded as requested, Option B)
class MouseModeEngine:
    def __init__(self,
                 screen_size=None,
                 median_k=7,
                 deadzone_px=2.8,
                 base_gain=1.18,
                 edge_boost=1.22,
                 corner_snap_dist_px=45,
                 lowpass_min_alpha=0.15,
                 lowpass_max_alpha=0.70,
                 velocity_alpha_scale=0.5,
                 history_maxlen=12):
        self.screen_w, self.screen_h = screen_size or pyautogui.size()

        self.median_k = median_k
        self.lowpass_min_alpha = lowpass_min_alpha
        self.lowpass_max_alpha = lowpass_max_alpha
        self.velocity_alpha_scale = velocity_alpha_scale
        self.history = deque(maxlen=history_maxlen)

        self.deadzone_px = deadzone_px
        self.base_gain = base_gain
        self.edge_boost = edge_boost
        self.corner_snap_dist_px = corner_snap_dist_px

        self.ema_x = None
        self.ema_y = None
        self.prev_time = None
        self.cursor_pos = (0.0, 0.0)
        self.cursor_ready = False
        self.last_move_time = None

    def _median_filter(self, pts):
        if not pts:
            return None
        xs = [p[0] for p in pts]
        ys = [p[1] for p in pts]
        k = min(self.median_k, len(xs))
        return float(np.median(xs[-k:])), float(np.median(ys[-k:]))

    def _compute_alpha(self, dt, v):
        if dt <= 0:
            dt = 1e-3
        v_norm = min(v / 2000.0, 1.0)
        alpha = self.lowpass_min_alpha + (
            self.lowpass_max_alpha - self.lowpass_min_alpha
        ) * (self.velocity_alpha_scale * v_norm + (1 - self.velocity_alpha_scale) * (dt / (dt + 0.02)))
        return max(min(alpha, self.lowpass_max_alpha), self.lowpass_min_alpha)

    def _edge_boost_factor(self, x, y):
        margin = min(min(x, self.screen_w - x), min(y, self.screen_h - y))
        thresh = min(self.screen_w, self.screen_h) * 0.1
        if margin < thresh:
            return 1 + (self.edge_boost - 1) * (1 - margin / thresh)
        return 1

    def _apply_deadzone(self, x, y):
        px, py = self.cursor_pos
        if math.hypot(x - px, y - py) <= self.deadzone_px:
            return (px, py), False
        return (x, y), True

    def update(self, pt, timestamp=None):
        """
        pt: normalized hand point (x_norm, y_norm) in [0..1], top-left origin
        timestamp: seconds (time.time())
        """
        if timestamp is None:
            timestamp = time.time()

        # normalized -> screen
        tx = float(np.clip(pt[0], 0.0, 1.0)) * (self.screen_w - 1)
        ty = float(np.clip(pt[1], 0.0, 1.0)) * (self.screen_h - 1)

        # push to history for median-filtering
        self.history.append((tx, ty, timestamp))
        med = self._median_filter(list(self.history))
        if med is None:
            return
        tx, ty = med

        # dt and velocity
        if self.prev_time is None:
            dt = 1.0 / 30.0
        else:
            dt = max(1e-3, timestamp - self.prev_time)
        self.prev_time = timestamp

        prev_x, prev_y = self.cursor_pos
        velocity = math.hypot(tx - prev_x, ty - prev_y) / max(dt, 1e-6)

        # adaptive alpha and gain
        alpha = self._compute_alpha(dt, velocity)
        gain = self.base_gain * self._edge_boost_factor(prev_x, prev_y)
        gain *= (1.0 + min(velocity / 2500.0, 0.6))

        # EMA smoothing
        if self.ema_x is None:
            self.ema_x, self.ema_y = tx, ty
        else:
            self.ema_x = alpha * tx + (1.0 - alpha) * self.ema_x
            self.ema_y = alpha * ty + (1.0 - alpha) * self.ema_y

        # apply gain around center
        cx = (self.screen_w - 1) / 2.0
        cy = (self.screen_h - 1) / 2.0
        out_x = cx + (self.ema_x - cx) * gain
        out_y = cy + (self.ema_y - cy) * gain

        # clamp to screen
        out_x = float(np.clip(out_x, 0.0, self.screen_w - 1))
        out_y = float(np.clip(out_y, 0.0, self.screen_h - 1))

        # deadzone
        (fx, fy), moved = self._apply_deadzone(out_x, out_y)
        if moved:
            self.cursor_pos = (fx, fy)
            self.last_move_time = timestamp
            self.cursor_ready = True

        # corner snap if within snap distance
        corners = [(0.0, 0.0), (self.screen_w - 1, 0.0), (0.0, self.screen_h - 1), (self.screen_w - 1, self.screen_h - 1)]
        fx, fy = self.cursor_pos
        in_corner = False
        for cx_t, cy_t in corners:
            if math.hypot(fx - cx_t, fy - cy_t) <= self.corner_snap_dist_px:
                in_corner = True
                corner_x, corner_y = cx_t, cy_t
                break
        if in_corner:
            dx = self.ema_x - fx  # Corrected logic here from original snippet
            dy = self.ema_y - fy
            movement_mag = math.hypot(dx, dy)
            ESCAPE_THRESHOLD = 25
            if movement_mag >= ESCAPE_THRESHOLD:
                self.cursor_pos = (fx + dx * 0.85, fy + dy * 0.85)
                self.cursor_ready = True
            else:
                self.cursor_pos = (corner_x, corner_y)
                self.cursor_ready = True
            
            BORDER_OFFSET = 7
            if fx <= 0:
                self.cursor_pos = (BORDER_OFFSET, self.cursor_pos[1])
            if fy <= 0:
                self.cursor_pos = (self.cursor_pos[0], BORDER_OFFSET)
            if fx >= self.screen_w - 1:
                self.cursor_pos = (self.screen_w - 1 - BORDER_OFFSET, self.cursor_pos[1])
            if fy >= self.screen_h - 1:
                self.cursor_pos = (self.cursor_pos[0], self.screen_h - 1 - BORDER_OFFSET)
            return
                
    def get_cursor(self):
        return (int(round(self.cursor_pos[0])), int(round(self.cursor_pos[1])))


# ==================================================================================================
# --- 1. NATIVE ANNOTATION CANVAS ---
# ==================================================================================================
class AnnotationCanvas:
    def __init__(self, root):
        self.root_ref = root
        self.screen_w, self.screen_h = pyautogui.size()

        self.win = tk.Toplevel(root)
        self.win.title("Annotation Overlay")
        self.win.geometry(f"{self.screen_w}x{self.screen_h}+0+0")
        self.win.overrideredirect(True)
        self.win.attributes("-topmost", True)

        # Colors tuned for visibility
        self.transparent_key = "white"
        self.opaque_color = "#FCF8F8"          # whiteboard
        self.pen_color = "#FF0000"             # bright red pen
        self.eraser_pointer_color = "#00FFFF"  # cyan eraser pointer

        if os.name == 'nt':
            try:
                self.win.attributes("-transparentcolor", self.transparent_key)
            except Exception:
                pass
            # slightly less transparent to make strokes more visible
            self.win.attributes("-alpha", 0.8)
        else:
            self.win.attributes("-alpha", 0.5)

        self.current_bg = self.transparent_key
        self.is_whiteboard = False

        self.canvas = tk.Canvas(
            self.win,
            width=self.screen_w,
            height=self.screen_h,
            bg=self.current_bg,
            highlightthickness=0
        )
        self.canvas.pack(fill="both", expand=True)

        self._stroke_items = []
        self.prev_x = None
        self.prev_y = None
        self.cursor_id = None  # used for both pen pointer and eraser pointer

        self.hide()

    def show(self):
        try:
            self.win.deiconify()
            self.win.lift()
        except Exception:
            pass

    def hide(self):
        try:
            self.win.withdraw()
        except Exception:
            pass

    def start_stroke(self):
        # Reset stroke continuity, but DO NOT hide the pointer.
        self.prev_x = None
        self.prev_y = None
        # self.cursor_id is kept so pointer circle stays visible

    def toggle_whiteboard_mode(self):
        self.is_whiteboard = not self.is_whiteboard
        if self.is_whiteboard:
            self.current_bg = self.opaque_color
            self.canvas.config(bg=self.opaque_color)
            if os.name == 'nt':
                self.win.attributes("-alpha", 1.0)
        else:
            self.current_bg = self.transparent_key
            self.canvas.config(bg=self.transparent_key)
            if os.name == 'nt':
                self.win.attributes("-alpha", 0.8)

        self.prev_x = None
        self.prev_y = None
        self._stroke_items.clear()
        return self.is_whiteboard

    def draw_point(self, x, y, mode="pen"):
        # Convert coords safely
        try:
            fx = float(x)
            fy = float(y)
        except Exception:
            fx, fy = int(x), int(y)

        # Remove previous pointer (pen or eraser) – it will be redrawn at new pos
        if self.cursor_id:
            try:
                self.canvas.delete(self.cursor_id)
            except Exception:
                pass
            self.cursor_id = None

        # ===================== POINTER ONLY (no drawing) =====================
        if mode == "pointer":
            r = 7
            try:
                self.cursor_id = self.canvas.create_oval(
                    fx - r, fy - r, fx + r, fy + r,
                    fill=self.pen_color,
                    outline="white",
                    width=2
                )
            except Exception:
                self.cursor_id = None
            self.prev_x, self.prev_y = fx, fy
            return

        # ===================== FIRST POINT OF STROKE =====================
        if self.prev_x is None:
            self.prev_x, self.prev_y = fx, fy
            # show pointer for the first point
            if mode == "pen":
                r = 7
                try:
                    self.cursor_id = self.canvas.create_oval(
                        fx - r, fy - r, fx + r, fy + r,
                        fill=self.pen_color,
                        outline="white",
                        width=2
                    )
                except Exception:
                    self.cursor_id = None
            elif mode == "eraser":
                r = 20
                try:
                    self.cursor_id = self.canvas.create_oval(
                        fx - r, fy - r, fx + r, fy + r,
                        fill=self.eraser_pointer_color,
                        outline="white",
                        width=2
                    )
                except Exception:
                    self.cursor_id = None
            return

        # ===================== PEN MODE (draw + pointer) =====================
        if mode == "pen":
            color = self.pen_color
            width = 6  # thicker for visibility

            # Draw stroke segment
            try:
                item = self.canvas.create_line(
                    float(self.prev_x), float(self.prev_y),
                    float(fx), float(fy),
                    fill=color,
                    width=width,
                    capstyle=tk.ROUND,
                    smooth=True
                )
                self._stroke_items.append(item)
            except Exception:
                pass

            # Draw pen pointer at tip
            r = 7
            try:
                self.cursor_id = self.canvas.create_oval(
                    fx - r, fy - r, fx + r, fy + r,
                    fill=self.pen_color,
                    outline="white",
                    width=2
                )
            except Exception:
                self.cursor_id = None

        # ===================== ERASER MODE (erase + big pointer) =====================
        elif mode == "eraser":
            width = 40
            if not self.is_whiteboard:
                # erase strokes in region
                try:
                    items = self.canvas.find_overlapping(
                        fx - width / 2, fy - width / 2,
                        fx + width / 2, fy + width / 2
                    )
                    for item in items:
                        if item != self.cursor_id:
                            try:
                                self.canvas.delete(item)
                                if item in self._stroke_items:
                                    self._stroke_items.remove(item)
                            except Exception:
                                pass
                except Exception:
                    pass
            else:
                # whiteboard: draw thick background-colored line
                color = self.current_bg
                try:
                    item = self.canvas.create_line(
                        float(self.prev_x), float(self.prev_y),
                        float(fx), float(fy),
                        fill=color,
                        width=width,
                        capstyle=tk.ROUND,
                        smooth=True
                    )
                    self._stroke_items.append(item)
                except Exception:
                    pass

            # eraser pointer
            r = width / 2
            try:
                self.cursor_id = self.canvas.create_oval(
                    fx - r, fy - r, fx + r, fy + r,
                    fill=self.eraser_pointer_color,
                    outline="white",
                    width=2
                )
            except Exception:
                self.cursor_id = None

        # update previous coords
        self.prev_x, self.prev_y = fx, fy

    def clear(self):
        try:
            self.canvas.delete("all")
        except Exception:
            pass
        self.cursor_id = None
        self._stroke_items.clear()

    def capture_stroke(self):
        try:
            self.win.update()
            x = self.win.winfo_rootx()
            y = self.win.winfo_rooty()
            w = self.win.winfo_width()
            h = self.win.winfo_height()
            img = pyautogui.screenshot(region=(x, y, w, h))
            return cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
        except Exception:
            return np.zeros((200, 200, 3), dtype=np.uint8)

# ==================================================================================================
# --- 3. HUD (Advanced Grid Layout) ---
# ==================================================================================================
class AdvancedHUD:
    def __init__(self, root):
        self.root = root
        self.width, self.height = 650, 250
        self.target_x, self.target_y = 20, 20
        self._anim_job = None

        self.hud = tk.Toplevel(root)
        self.hud.overrideredirect(True)
        self.hud.attributes("-topmost", True)
        self.hud.attributes("-alpha", 0.0)
        self.hud.geometry(f"{self.width}x{self.height}+{-self.width}+{self.target_y}")

        self.current_x = -self.width
        self.current_alpha = 0.0
        self.is_visible = False

        self._build_ui()
        self._enable_acrylic_blur()

        self.show(immediate=True)

    def _build_ui(self):
        outer = tk.Frame(self.hud, bg="#101010")
        outer.pack(fill="both", expand=True, padx=6, pady=6)

        top = tk.Frame(outer, bg="#101010")
        top.pack(fill="x")
        top.columnconfigure(1, weight=1)

        self.icon_label = tk.Label(
            top, text="🟢", bg="#101010", fg="#00FF88", font=("Segoe UI Emoji", 11)
        )
        self.icon_label.grid(row=0, column=0, padx=(5, 10), pady=3)

        self.hud_mode_label = tk.Label(
            top, text="MODE: Init...", fg="#E0E0E0",
            bg="#101010", font=("Segoe UI", 10, "bold"), anchor="w"
        )
        self.hud_mode_label.grid(row=0, column=1, sticky="w")

        self.fps_label = tk.Label(
            top, text="FPS: --", fg="#A0A0A0", bg="#101010",
            font=("Segoe UI", 10)
        )
        self.fps_label.grid(row=0, column=2, padx=10)

        status_frame = tk.Frame(outer, bg="#101010")
        status_frame.pack(fill="x", pady=3)
        status_frame.columnconfigure(1, weight=1)

        tk.Label(
            status_frame, text="STATUS:", fg="#FFD700", bg="#101010",
            font=("Segoe UI", 10, "bold")
        ).grid(row=0, column=0, padx=5, sticky="w")

        self.hud_status_label = tk.Label(
            status_frame, text="Hand Not Detected",
            fg="#FFD700", bg="#101010",
            font=("Segoe UI", 10), anchor="w"
        )
        self.hud_status_label.grid(row=0, column=1, sticky="w")

        context_frame = tk.Frame(outer, bg="#101010")
        context_frame.pack(fill="x", pady=3)
        context_frame.columnconfigure(1, weight=1)

        tk.Label(
            context_frame, text="AI CONTEXT:", fg="#00A0FF", bg="#101010",
            font=("Segoe UI", 10, "bold")
        ).grid(row=0, column=0, padx=5, sticky="w")

        self.hud_context_label = tk.Label(
            context_frame, text="TEXT_SLIDE",
            fg="#00A0FF", bg="#101010",
            font=("Segoe UI", 10), anchor="w"
        )
        self.hud_context_label.grid(row=0, column=1, sticky="w")

        bottom = tk.Frame(outer, bg="#101010")
        bottom.pack(fill="x", pady=5)

        self.hand_label = tk.Label(
            bottom, text="HAND: Not detected", fg="#FF5555", bg="#101010"
        )
        self.hand_label.pack(side="left")

        self.conf_bar_width = 200
        self.conf_canvas = tk.Canvas(
            bottom, width=self.conf_bar_width, height=8,
            bg="#202020", bd=0, highlightthickness=0
        )
        self.conf_canvas.pack(side="right")

    def _enable_acrylic_blur(self):
        try:
            hwnd = self.hud.winfo_id()

            class ACCENTPOLICY(ctypes.Structure):
                fields = [
                    ("AccentState", ctypes.c_int),
                    ("AccentFlags", ctypes.c_int),
                    ("GradientColor", ctypes.c_int),
                    ("AnimationId", ctypes.c_int),
                ]

            class WINDOWCOMPOSITIONATTRIBDATA(ctypes.Structure):
                fields = [
                    ("Attribute", ctypes.c_int),
                    ("Data", ctypes.c_void_p),
                    ("SizeOfData", ctypes.c_size_t),
                ]

            accent = ACCENTPOLICY(
                4, 0,
                (180 << 24) | (0x15 << 16) | (0x15 << 8) | 0x15,
                0
            )
            data = WINDOWCOMPOSITIONATTRIBDATA(
                19,
                ctypes.addressof(accent),
                ctypes.sizeof(accent)
            )
            ctypes.windll.user32.SetWindowCompositionAttribute(
                hwnd, ctypes.byref(data)
            )
        except Exception:
            pass

    def _cancel_anim(self):
        if self._anim_job:
            try:
                self.hud.after_cancel(self._anim_job)
            except Exception:
                pass
            self._anim_job = None

    def show(self, immediate=False):
        self._cancel_anim()
        self.is_visible = True
        try:
            self.hud.deiconify()
            self.hud.lift()
        except Exception:
            pass

        if immediate:
            self.current_x = self.target_x
            self.current_alpha = 1.0
            try:
                self.hud.geometry(
                    f"{self.width}x{self.height}+{int(self.current_x)}+{self.target_y}"
                )
                self.hud.attributes("-alpha", self.current_alpha)
            except Exception:
                pass
            return

        self._animate_in()

    def _animate_in(self):
        dist = self.target_x - self.current_x
        step = max(12, int(abs(dist) * 0.25))
        alpha_step = 0.18

        if self.current_x < self.target_x:
            self.current_x += step
            if self.current_x > self.target_x:
                self.current_x = self.target_x

        if self.current_alpha < 0.95:
            self.current_alpha += alpha_step
        else:
            self.current_alpha = 1.0

        try:
            self.hud.geometry(
                f"{self.width}x{self.height}+{int(self.current_x)}+{self.target_y}"
            )
            self.hud.attributes(
                "-alpha", max(0.0, min(1.0, self.current_alpha))
            )
        except Exception:
            pass

        if self.current_x != self.target_x or self.current_alpha < 1.0:
            self._anim_job = self.hud.after(16, self._animate_in)
        else:
            self._anim_job = None

    def hide(self, immediate=False):
        self._cancel_anim()
        self.is_visible = False

        if immediate:
            self.current_x = -self.width
            self.current_alpha = 0.0
            try:
                self.hud.geometry(
                    f"{self.width}x{self.height}+{int(self.current_x)}+{self.target_y}"
                )
                self.hud.attributes("-alpha", 0.0)
                self.hud.withdraw()
            except Exception:
                pass
            return

        self._animate_out()

    def _animate_out(self):
        target_hide_x = -self.width
        dist = self.current_x - target_hide_x
        step = max(12, int(abs(dist) * 0.25))
        alpha_step = 0.18

        if self.current_x > target_hide_x:
            self.current_x -= step
            if self.current_x < target_hide_x:
                self.current_x = target_hide_x

        if self.current_alpha > 0.05:
            self.current_alpha -= alpha_step
        else:
            self.current_alpha = 0.0

        try:
            self.hud.geometry(
                f"{self.width}x{self.height}+{int(self.current_x)}+{self.target_y}"
            )
            self.hud.attributes(
                "-alpha", max(0.0, min(1.0, self.current_alpha))
            )
        except Exception:
            pass

        if self.current_x != target_hide_x or self.current_alpha > 0.0:
            self._anim_job = self.hud.after(16, self._animate_out)
        else:
            try:
                self.hud.withdraw()
            except Exception:
                pass
            self._anim_job = None

    def update_hud(self, mode, status, fps, confidence, hand_detected, context):
        try:
            self.hud_mode_label.config(text=f"MODE: {mode}")
            disp = status.replace("STATUS:", "").strip() if "STATUS:" in str(status) else status
            self.hud_status_label.config(text=f"STATUS: {disp}")
            self.fps_label.config(text=f"FPS: {fps:.1f}")
            self.hud_context_label.config(text=f"AI CONTEXT: {context.upper()}")
            self.hand_label.config(
                text="HAND: Detected" if hand_detected else "HAND: Not detected",
                fg="#00FF88" if hand_detected else "#FF5555"
            )

            self.conf_canvas.delete("bar")
            w = int(self.conf_bar_width * max(0, min(1, confidence)))
            color = "#FF5555" if confidence < 0.4 else "#FFBB33" if confidence < 0.75 else "#00CC66"
            self.conf_canvas.create_rectangle(
                0, 0, w, 8, fill=color, outline="", tags=("bar",)
            )
        except Exception:
            pass

# ==================================================================================================
# --- 4/5. GESTURE CONTROLLER ---
# ==================================================================================================
class GestureController:
    def __init__(self, ui_queue, annotation_predictor=None):
        self.ui_queue = ui_queue
        self.stop_event = threading.Event()

        import mediapipe as mp
        self.mp_hands = mp.solutions.hands
        self.mp_drawing = mp.solutions.drawing_utils

        self.hands = self.mp_hands.Hands(
            static_image_mode=False,
            max_num_hands=1,
            min_detection_confidence=0.7,
            min_tracking_confidence=0.7
        )

        self.screen_w, self.screen_h = pyautogui.size()
        self.cap = cv2.VideoCapture(0)
        if not self.cap.isOpened():
            print("CRITICAL: No camera found.")
            self.stop_event.set()

        self.current_mode = "Mouse Mode"
        self.current_slide_context = "text_slide"
        self.status_text = "STATUS: Initializing..."
        self.confirmed_gesture = "NONE"
        self.gesture_buffer = deque(maxlen=6)

        self.last_presentation_action_time = 0
        self.presentation_action_cooldown = 0.5
        self.last_whiteboard_toggle_time = 0
        self.mode_switch_cooldown = 0

        # Slideshow open-palm timing
        self.last_palm_toggle_time = 0.0
        self.palm_quick_cooldown = 0.8     # quick tap cooldown
        self.palm_long_hold_time = 1.5     # seconds to end slideshow
        self.palm_hold_start = None

        # One-shot guard (for PINCH, FIST, TWO_FINGERS)
        self.last_trigger_gesture = None

        # HUD toggle (FOUR_FINGERS)
        self.four_finger_last = 0
        self.four_finger_cooldown = 0.7
        self.last_hud_toggle_gesture = None

        # Mouse smoothing
        self.smoothening = 5
        self.prev_x = float(self.screen_w / 2)
        self.prev_y = float(self.screen_h / 2)
        self.zoom_active = False
        self.last_frame_time = 0
        self.current_fps = 0
        self.overlay_ref = None
        self.slideshow_running = False
        self.hud_hidden = False
        self.hud_ref = None

        self.kalman_x = self._create_kalman()
        self.kalman_y = self._create_kalman()
        self.ma_window = deque(maxlen=7)
        self.ema_x = None
        self.ema_y = None
        self.ema_alpha = 0.18
        self.dead_zone_threshold = 1.8
        self._dead_zone_time = 0.45
        self.dead_zone_lock = False
        self._dead_zone_since = None
        self.fast_speed_threshold = 90.0
        self.prediction_factor = 0.12
        self.edge_push = 12

        self.annotation_predictor = annotation_predictor

        # Pen state for annotation mode
        self.pen_down = True  # start as pen touching (drawing) by default

        # === PRO Mouse engine (embedded) ===
        self.mouse_engine = MouseModeEngine((self.screen_w, self.screen_h))

    def _create_kalman(self):
        kf = cv2.KalmanFilter(2, 1)
        kf.measurementMatrix = np.array([[1., 0.]], np.float32)
        kf.transitionMatrix = np.array([[1., 1.], [0., 1.]], np.float32)
        kf.processNoiseCov = np.array([[1e-2, 0.], [0., 1e-2]], np.float32)
        kf.measurementNoiseCov = np.array([[1e-1]], np.float32)
        kf.errorCovPost = np.eye(2, dtype=np.float32)
        return kf

    def get_finger_status(self, hand, handed):
        fingers = []
        tip = [4, 8, 12, 16, 20]

        if handed == "Right":
            fingers.append(hand.landmark[tip[0]].x < hand.landmark[tip[0] - 1].x)
        else:
            fingers.append(hand.landmark[tip[0]].x > hand.landmark[tip[0] - 1].x)

        for i in range(1, 5):
            fingers.append(hand.landmark[tip[i]].y < hand.landmark[tip[i] - 2].y)

        return fingers

    def get_landmark_distance(self, lm1, lm2, w, h):
        x1, y1 = int(lm1.x * w), int(lm1.y * h)
        x2, y2 = int(lm2.x * w), int(lm2.y * h)
        return math.hypot(x2 - x1, y2 - y1)

    def detect_gesture(self, hand, w, h, handed):
        f = self.get_finger_status(hand, handed)
        scale = self.get_landmark_distance(hand.landmark[0], hand.landmark[5], w, h)
        if scale == 0:
            scale = 1

        dist_ti = self.get_landmark_distance(hand.landmark[4], hand.landmark[8], w, h)
        dist_tm = self.get_landmark_distance(hand.landmark[4], hand.landmark[12], w, h)

        if dist_ti < scale * 0.5 and dist_tm < scale * 0.5 and not f[3] and not f[4]:
            return "DOUBLE_PINCH"

        if f[1] and f[2] and f[3] and not f[0] and not f[4]:
            return "THREE_FINGERS"

        if f[4] and not any(f[:4]):
            return "PINKY_ONLY"

        if f[0] and f[1] and f[4] and not f[2] and not f[3]:
            return "THUMB_INDEX_PINKY"

        if f[1] and f[2] and f[3] and f[4] and not f[0]:
            return "FOUR_FINGERS"

        if all(f):
            return "FIVE_OPEN_PALM"

        if f[1] and f[2] and not any([f[0], f[3], f[4]]):
            return "TWO_FINGERS"

        if not any(f):
            return "FIST"

        if f[1] and not any([f[2], f[3], f[4]]):
            dist_pinch = self.get_landmark_distance(hand.landmark[4], hand.landmark[8], w, h)
            if dist_pinch < scale * 0.4:
                return "PINCH_CLICK"
            return "CURSOR"

        return "NONE"

    def perform_action(self, gesture, hand, w, h):
        # One-shot gestures handled here (FIVE_OPEN_PALM is handled separately with timing)
        one_shot_gestures = {"PINCH_CLICK", "FIST", "TWO_FINGERS"}

        # Unlock one-shot when relaxed / cursor
        if gesture in ("NONE", "CURSOR"):
            self.last_trigger_gesture = None

        # Block continuous firing of the same gesture
        if gesture in one_shot_gestures and gesture == self.last_trigger_gesture:
            return

        did_action = False

        if not self.zoom_active:
            # ===================== MOUSE MODE =====================
            if self.current_mode == "Mouse Mode":
                if gesture == "PINCH_CLICK":
                    try:
                        pyautogui.click()
                    except Exception:
                        pass
                    self.status_text = "Action: Left Click"
                    did_action = True

                elif gesture == "FIST":
                    try:
                        pyautogui.rightClick()
                    except Exception:
                        pass
                    self.status_text = "Action: Right Click"
                    did_action = True

            # ===================== PRESENTATION MODE =====================
            elif self.current_mode == "Presentation Mode":
                now = time.time()

                PALM_TAP_MAX = 0.60      # tap < 0.6 sec
                PALM_HOLD_MIN = 1.20     # hold > 1.2 sec
                PALM_COOLDOWN = 1.0      # cooldown for palm trigger

                did_action = False

                # ----------------------------------------------------
                # FIVE_OPEN_PALM -> Start (tap) / End (hold) slideshow
                # ----------------------------------------------------
                if gesture == "FIVE_OPEN_PALM":

                    # Start or continue timing the palm hold
                    if self.palm_hold_start is None:
                        self.palm_hold_start = now

                    hold_time = now - self.palm_hold_start

                    # Ignore events during cooldown
                    if now - self.last_palm_toggle_time < PALM_COOLDOWN:
                        pass

                    # ---------- TAP -> START SLIDESHOW ----------
                    elif (not self.slideshow_running
                          and hold_time > 0.5):

                        try:
                            pyautogui.press("f5")
                            self.slideshow_running = True
                            self.status_text = "Start Slideshow (Palm)"
                        except Exception:
                            pass

                        self.last_palm_toggle_time = now
                        self.palm_hold_start = None
                        self.palm_released = False
                        did_action = True

                    # ---------- LONG HOLD -> END SLIDESHOW ----------
                    elif self.slideshow_running and hold_time >= PALM_HOLD_MIN:

                        try:
                            pyautogui.press("esc")
                            self.slideshow_running = False
                            self.status_text = "End Slideshow (Palm Hold)"
                        except Exception:
                            pass

                        self.last_palm_toggle_time = now
                        self.palm_hold_start = None
                        self.palm_released = False
                        did_action = True

                # ----------------------------------------------------
                # PALM GONE (gesture != FIVE_OPEN_PALM)
                # ----------------------------------------------------
                else:
                    # Palm disappeared -> ready for next tap
                    if self.palm_hold_start is not None:
                        self.palm_released = True

                    self.palm_hold_start = None

                    # ------------------------------------------------
                    # TWO_FINGERS -> Next slide
                    # ------------------------------------------------
                    if now - self.last_presentation_action_time >= self.presentation_action_cooldown:

                        if gesture == "TWO_FINGERS":
                            try:
                                pyautogui.press("right")
                            except Exception:
                                pass
                            self.status_text = "Slide Next"
                            did_action = True

                        # ------------------------------------------------
                        # FIST -> Previous slide
                        # ------------------------------------------------
                        elif gesture == "FIST":
                            try:
                                pyautogui.press("left")
                            except Exception:
                                pass
                            self.status_text = "Slide Previous"
                            did_action = True

                        if did_action:
                            self.last_presentation_action_time = now

            # ===================== ANNOTATION MODE =====================
            elif self.current_mode == "Annotation Mode":
                if gesture == "PINCH_CLICK":
                    # Toggle pen state: down -> up, up -> down
                    if self.pen_down:
                        # LIFT PEN
                        self.pen_down = False
                        self.status_text = "Pen Lifted"
                        if self.overlay_ref:
                            # end current stroke, but pointer stays (start_stroke doesn't delete pointer)
                            self.overlay_ref.start_stroke()
                            if self.annotation_predictor:
                                stroke = self.overlay_ref.capture_stroke()
                                try:
                                    label, conf = self.annotation_predictor.predict(stroke)
                                    self.status_text = f"Recognized: {label} ({conf:.2f})"
                                except Exception:
                                    pass
                    else:
                        # PUT PEN DOWN
                        self.pen_down = True
                        self.status_text = "Pen Down"
                    did_action = True

                elif gesture == "FIST":
                    self.status_text = "Action: Clear Board"
                    if self.overlay_ref:
                        self.overlay_ref.clear()
                    did_action = True

                elif gesture == "FIVE_OPEN_PALM":
                    now = time.time()
                    if now - self.last_whiteboard_toggle_time > 1.5:
                        if self.overlay_ref:
                            is_wb = self.overlay_ref.toggle_whiteboard_mode()
                            self.status_text = "Whiteboard Mode" if is_wb else "Overlay Mode"
                        self.last_whiteboard_toggle_time = now
                        did_action = True

        # Update one-shot lock if action occurred
        if did_action and gesture in one_shot_gestures:
            self.last_trigger_gesture = gesture

    def run(self):
        while not self.stop_event.is_set():
            ret, frame = self.cap.read()
            if not ret:
                break

            curr_time = time.time()
            fps = 1 / (curr_time - self.last_frame_time) if self.last_frame_time > 0 else 0
            self.last_frame_time = curr_time
            self.current_fps = fps

            frame = cv2.flip(frame, 1)
            h, w, _ = frame.shape

            rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = self.hands.process(rgb)

            detected = False
            confidence = 0.0

            if results.multi_hand_landmarks:
                detected = True

                for hand_landmarks in results.multi_hand_landmarks:
                    self.mp_drawing.draw_landmarks(
                        frame,
                        hand_landmarks,
                        self.mp_hands.HAND_CONNECTIONS
                    )

                hand = results.multi_hand_landmarks[0]
                handedness = results.multi_handedness[0].classification[0].label
                confidence = results.multi_handedness[0].classification[0].score

                gesture = self.detect_gesture(hand, w, h, handedness)

                self.gesture_buffer.append(gesture)
                if len(self.gesture_buffer) == self.gesture_buffer.maxlen:
                    most_common = max(
                        set(self.gesture_buffer),
                        key=self.gesture_buffer.count
                    )
                    if self.gesture_buffer.count(most_common) >= 4:
                        self.confirmed_gesture = most_common

                idx = hand.landmark[8]

                # ------------------ Mouse Mode cursor ------------------
                if self.current_mode == "Mouse Mode" and self.confirmed_gesture == "CURSOR":
                    # Use embedded PRO MouseModeEngine
                    x_norm = idx.x
                    y_norm = idx.y
                    try:
                        self.mouse_engine.update((x_norm, y_norm), time.time())
                        if self.mouse_engine.cursor_ready:
                            px, py = self.mouse_engine.get_cursor()
                            pyautogui.moveTo(px, py, duration=0)
                            self.prev_x, self.prev_y = float(px), float(py)
                            self.status_text = f"STATUS: Moving Cursor (PRO)"
                    except Exception:
                        # fallback to previous simple mapping if anything fails
                        try:
                            cx_raw = idx.x * self.screen_w
                            cy_raw = idx.y * self.screen_h
                            pyautogui.moveTo(cx_raw, cy_raw)
                            self.prev_x, self.prev_y = cx_raw, cy_raw
                            self.status_text = "STATUS: Moving Cursor (fallback)"
                        except Exception:
                            pass

                # ------------------ Annotation drawing ------------------
                elif self.current_mode == "Annotation Mode":
                    cx = idx.x * self.screen_w
                    cy = idx.y * self.screen_h

                    if self.ema_x is None:
                        self.ema_x, self.ema_y = cx, cy

                    self.ema_x = (1 - self.ema_alpha) * self.ema_x + self.ema_alpha * cx
                    self.ema_y = (1 - self.ema_alpha) * self.ema_y + self.ema_alpha * cy

                    if self.confirmed_gesture == "CURSOR":
                        if self.overlay_ref:
                            if self.pen_down:
                                # draw + pointer
                                self.overlay_ref.draw_point(self.ema_x, self.ema_y, mode="pen")
                                self.status_text = "Drawing"
                            else:
                                # pointer only (pen lifted)
                                self.overlay_ref.draw_point(self.ema_x, self.ema_y, mode="pointer")
                                self.status_text = "Pen Hover"

                    elif self.confirmed_gesture == "DOUBLE_PINCH":
                        if self.overlay_ref:
                            self.overlay_ref.draw_point(self.ema_x, self.ema_y, mode="eraser")
                            self.status_text = "Erasing"

                # ------------------ MODE SWITCHING & HUD ------------------
                now = time.time()

                # FOUR_FINGERS: HUD toggle (simple one-shot)
                if self.confirmed_gesture == "FOUR_FINGERS":
                    if (self.last_hud_toggle_gesture != "FOUR_FINGERS" and
                            now - self.four_finger_last > self.four_finger_cooldown):
                        if self.hud_ref:
                            if self.hud_ref.is_visible:
                                self.hud_ref.hide(immediate=False)
                                self.hud_hidden = True
                            else:
                                self.hud_ref.show()
                                self.hud_hidden = False
                        self.four_finger_last = now
                        self.last_hud_toggle_gesture = "FOUR_FINGERS"
                else:
                    if self.last_hud_toggle_gesture == "FOUR_FINGERS":
                        self.last_hud_toggle_gesture = None

                # Mode switching (with cooldown)
                if now - self.mode_switch_cooldown > 1.5:
                    switched = False

                    if self.confirmed_gesture == "THUMB_INDEX_PINKY":
                        self.current_mode = "Mouse Mode"
                        self.status_text = "Switched to Mouse Mode"
                        self.toggle_overlay_visibility()
                        switched = True

                    elif self.confirmed_gesture == "TWO_FINGERS":
                        if self.current_mode != "Presentation Mode":
                            self.current_mode = "Presentation Mode"
                            self.status_text = "Switched to Presentation Mode"
                            self.toggle_overlay_visibility()
                            switched = True

                    elif self.confirmed_gesture == "THREE_FINGERS":
                        if self.current_mode != "Annotation Mode":
                            self.current_mode = "Annotation Mode"
                            self.status_text = "Switched to Annotation Mode"
                            self.toggle_overlay_visibility()
                            switched = True

                    if switched:
                        self.mode_switch_cooldown = now

                # Perform actions
                self.perform_action(self.confirmed_gesture, hand, w, h)

            else:
                detected = False
                self.status_text = "STATUS: No Hand Detected"
                self.gesture_buffer.clear()
                self.last_trigger_gesture = None
                self.palm_hold_start = None
                if self.overlay_ref:
                    self.overlay_ref.start_stroke()

            try:
                self.ui_queue.put_nowait(
                    (
                        frame,
                        self.current_mode,
                        self.status_text,
                        self.current_fps,
                        confidence,
                        detected,
                        self.current_slide_context,
                    )
                )
            except queue.Full:
                pass

            time.sleep(0.01)

        self.cap.release()

    def toggle_overlay_visibility(self):
        if not self.overlay_ref:
            return
        if self.current_mode == "Annotation Mode":
            self.overlay_ref.show()
        else:
            self.overlay_ref.hide()

    def stop(self):
        self.stop_event.set()

# ==================================================================================================
# --- 6. MAIN APPLICATION ---
# ==================================================================================================
class App:
    def __init__(self, root):
        self.root = root
        self.root.title("Gesture Control Panel - PRO")
        self.is_closing = False

        self.camera_label = tk.Label(root, bg="black")
        self.camera_label.pack(fill="both", expand=True)

        self.screen_w, self.screen_h = pyautogui.size()

        self.popup = tk.Toplevel(root)
        self.popup.withdraw()
        self.popup.overrideredirect(True)
        self.popup.attributes("-topmost", True)

        self.popup_w, self.popup_h = 400, 75
        self.popup_label = tk.Label(
            self.popup,
            text="",
            font=("Helvetica", 10, "bold"),
            bg="gray20",
            fg="white",
            padx=14,
            pady=10,
        )
        self.popup_label.pack(fill="both", expand=True)
        self._popup_hide_job = None
        self.last_popup_text = ""

        self.hud = AdvancedHUD(root)
        self.annotation_overlay = AnnotationCanvas(root)

        self.ui_queue = queue.Queue(maxsize=2)
        self.annotation_predictor = None  # plug your predictor if you have one

        self.gesture_controller = GestureController(
            self.ui_queue, annotation_predictor=self.annotation_predictor
        )
        self.gesture_controller.overlay_ref = self.annotation_overlay
        self.gesture_controller.hud_ref = self.hud

        self.gesture_thread = threading.Thread(
            target=self.gesture_controller.run, daemon=True
        )
        self.gesture_thread.start()

        self.root.protocol("WM_DELETE_WINDOW", self.on_closing)
        self.root.after(20, self.poll_ui_queue)

    def poll_ui_queue(self):
        if self.is_closing:
            return
        try:
            frame, mode, status, fps, confidence, hand_detected, context = self.ui_queue.get_nowait()
            if frame is not None:
                self.update_ui_frame(frame)
            self.update_hud_status(mode, status, fps, confidence, hand_detected, context)

            if any(key in status for key in ["Action:", "Switched", "HUD", "Mode", "Recognized:", "Slide", "Slideshow", "Pen"]):
                if status != self.last_popup_text:
                    self.show_popup(status, "#27ae60")
                    self.last_popup_text = status
            else:
                self.last_popup_text = ""
        except queue.Empty:
            pass

        self.root.after(20, self.poll_ui_queue)

    def update_ui_frame(self, frame):
        try:
            img = ImageTk.PhotoImage(
                Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
            )
            self.camera_label.imgtk = img
            self.camera_label.configure(image=img)
        except Exception:
            pass

    def update_hud_status(self, mode, status, fps, confidence, hand_detected, context):
        self.hud.update_hud(
            mode=mode,
            status=status,
            fps=fps,
            confidence=confidence,
            hand_detected=hand_detected,
            context=context,
        )

    def show_popup(self, text, color="gray25", duration_ms=1300):
        def _do_popup():
            if self._popup_hide_job:
                self.root.after_cancel(self._popup_hide_job)

            popup_x = self.screen_w - self.popup_w - 25
            popup_y = 120
            self.popup.geometry(f"{self.popup_w}x{self.popup_h}+{popup_x}+{popup_y}")
            self.popup_label.config(text=text, bg=color)
            self.popup.deiconify()
            self.popup.lift()
            self._popup_hide_job = self.root.after(duration_ms, self._hide_popup)

        self.root.after(0, _do_popup)

    def _hide_popup(self):
        self.popup.withdraw()
        self._popup_hide_job = None

    def on_closing(self, event=None):
        if self.is_closing:
            return
        self.is_closing = True

        self.gesture_controller.stop()
        time.sleep(0.2)

        try:
            self.annotation_overlay.win.destroy()
        except Exception:
            pass
        try:
            self.hud.hud.destroy()
        except Exception:
            pass
        try:
            self.popup.destroy()
        except Exception:
            pass
        try:
            self.root.destroy()
        except Exception:
            pass

if __name__ == "__main__":
    root = tk.Tk()
    root.geometry("800x600")
    app = App(root)
    root.mainloop()